In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
import pickle

In [34]:
test_set = pd.read_csv("data/test_set.csv")
train_set = pd.read_csv("data/train_set.csv")

In [42]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))

train_binary_scaled = pd.DataFrame(min_max_scaler.fit_transform(train_set.values), columns=train_set.columns, index=train_set.index)
test_binary_scaled = pd.DataFrame(min_max_scaler.transform(test_set.values), columns=test_set.columns, index=test_set.index)

X_train = train_binary_scaled.drop("Diabetes_012", axis = 1)
y_train = train_binary_scaled["Diabetes_012"]

X_test = test_binary_scaled.drop("Diabetes_012", axis = 1)
y_test = test_binary_scaled["Diabetes_012"]

In [21]:
pd.set_option('display.max_columns', 25)
X_train.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,0.0,1.0,0.407407,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.833333,1.0,1.000000
1,0.0,0.0,0.0,0.703704,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.50,0.4,0.0,0.0,1.0,0.500000,0.6,1.000000
2,1.0,0.0,1.0,0.703704,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.75,0.0,0.0,0.0,0.0,0.583333,0.6,0.285714
3,1.0,1.0,1.0,0.444444,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.50,0.0,0.0,1.0,1.0,0.916667,1.0,0.285714
4,1.0,1.0,1.0,0.777778,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.75,0.4,0.2,1.0,0.0,0.833333,0.2,0.000000


### Badane cechy:
1. **HighBP**: 
    - 0 - brak wysokiego ciśnienia krwi (lub brak świadomości)
    - 1 - wysokie ciśnienie krwi

2. **HighChol**: 
    - 0 - brak wysokiego cholesterolu (lub brak świadomości)
    - 1 - wysoki cholesterol

3. **CholCheck**: 
    - 0 - brak badania cholesterolu w ciągu ostatnich 5 lat
    - 1 - badanie cholesterolu w ciągu ostatnich 5 lat
       
4. **BMI**: Body Mass Index zaokrąglony do INT

5. **Smoker**: 
    - 0 - wypalił(a) mniej niż 100 papierosów w życiu (niepalący)
    - 1 - więcej niż 100 papierosów

6. **Stroke**: 
    - 0 - nie miał(a) udaru mózgu
    - 1 - był udar

7. **HeartDiseaseorAttack**: 
    - 0 - nie miał(a) choroby wieńcowej ani zawału serca
    - 1 - wystąpiła conajmniej jedna z tych przypadłości

8. **PhysActivity**: 
    - 0 - nie uprawiał(a) sportu przez ostatnie 30 dni (nie licząc pracy)
    - 1 - uprawiał(a) sport

9. **Fruits**: 
    - 0 - spożywa owoce mniej niż raz dziennie
    - 1 - conajmiej raz dziennie

10. **Veggies**: 
    - 0 - spożywa warzywa mniej niż raz dziennie
    - 1 - conajmiej raz dziennie

11. **HvyAlcoholConsumption**: 
    - 0 - spożywa mniej niż 14 drinków tygodniowo (mężczyźni) lub mniej niż 7 tygodniowo (kobiety)
    - 1 - spożywa więcej alkoholu

12. **AnyHealthcare**: 
    - 0 - nie posiada żadnej formy ubezpieczenia zdrowotnego
    - 1 - posiada ubezpieczenie zdrowotne, korzysta z prywatnej organizacji ochrony zdrowia itp.

13. **NoDocbcCost**: Czy zdarzyła się sytuacja w ciągu ostatnich 12 miesięcy, w której osoba potrzebowała udać się do lekarza, ale nie mogła ze względu na wysokie koszty?
    - 0 - nie
    - 1 - tak

14. **GenHlth**: Subiektywna ocena ogólnego zdrowia w skali 1-5
    - 1 - świetnie, 
    - 2 - bardzo dobrze, 
    - 3 - dobrze, 
    - 4 - średnio, 
    - 5 - słabo

15. **MentHlth**: Na przestrzeni ostatnich 30 dni, ile dni było takich, że osoba oceniłaby swoje zdrowie psychiczne negatywnie (włączając stres, depresję, problemy emocjonalne)?
    - 0 - dobre zdrowie psychiczne
    - ...
    - 5 - bardzo złe zdrowie psychiczne

16. **PhysHlth**: Na przestrzeni ostatnich 30 dni, ile dni było takich, że osoba oceniłaby swoje zdrowie fizyczne negatywnie (włączając fizyczne choroby, wypadki)?
    - 0 - dobre zdrowie fizyczne
    - ...
    - 5 - bardzo złe zdrowie fizyczne

17. **DiffWalk**: 
    - 0 - osoba nie ma problemów z chodzeniem po schodach
    - 1 - ma problemy z chodzeniem po schodach

18. **Sex**: 
    - 0 - kobieta
    - 1 - mężczyzna

19. **Age**: 
    - Wiek pogrupowany w przedziały o interwale około 5 lat, inkluzywnie 
    - 1: wiek 18-24
    - 2: wiek 25-29
    - ...
    - 13: wiek >80

20. **Education**: 
    - 1 - brak szkoły lub tylko przedszkole
    - 2 - wykształcenie podstawowe
    - 3 - nieukończona (ale rozpoczęta) szkoła średnia
    - 4 - wykształcenie średnie
    - 5 - nieukończone (ale rozpoczęte) studia
    - 6 - wykształcenie wyższe

21. **Income**: 
    - Dochód roczny w $ zgrupowany w przedziały
    - 1 - (<10 000)
    - 2 - (10 000, 15 000)
    - ...
    - 5 - (25 000, 35 000)
    - ...
    - 8 - (>75 000)


In [76]:
# Twój słownik zawierający dane
dane_1 = {
    'Dummy': 0,
    
    
    'HighBP': 0,
    'HighChol': 0,
    'CholCheck': 0,
    'BMI': 23.6,
    'Smoker': 0,
    'Stroke': 0,
    'HeartDiseaseorAttack': 0,
    'PhysActivity': 1,
    'Fruits': 1,
    'Veggies': 1,
    'HvyAlcoholConsump': 0,
    'AnyHealthcare': 1,
    'NoDocbcCost': 0,
    'GenHlth': 1,
    'MentHlth': 0,
    'PhysHlth': 0,
    'DiffWalk': 0,
    'Sex': 1,
    'Age': 1,
    'Education': 3,
    'Income': 1
}

# Tworzenie DataFrame
df1 = pd.DataFrame(dane_1, index = [0])

# Wyświetlenie DataFrame
df1

,Dummy,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,0,0,0,23.6,0,0,0,1,1,1,0,1,0,1,0,0,0,1,1,3,1


In [77]:
df1_scaled = pd.DataFrame(min_max_scaler.transform(df1.values), columns=df1.columns)
df1_scaled = df1_scaled.iloc[:, 1:22]
df1_scaled

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,0.0,0.0,0.355556,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,-0.2,-0.2,0.0,1.0,0.0,0.4,0.0


In [57]:
gboost = GradientBoostingClassifier(subsample=1, n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features='sqrt', max_depth=3, learning_rate=0.1)
gboost.fit(X_train, y_train)

GradientBoostingClassifier(max_features='sqrt', min_samples_leaf=2,
                           n_estimators=200, subsample=1)

In [78]:
print(gboost.predict_proba(df1_scaled)[:,1] * 100, "%")

[0.59942776] %
